# 一、数据清洗

In [1]:
import pymongo
# MongoDB连接配置
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
convoinsight_db = mongo_client["convoinsight"]
conversations_collection = convoinsight_db["conversations"]

## 1. 清洗会话用户情感

In [5]:
#查看用户会话的sentiments的所有情感类型值和频数
# conversations_collection.find_one()
# 查询所有用户消息
from collections import Counter
user_messages = conversations_collection.aggregate([
    {"$unwind": "$messages"},
    {"$match": {"messages.type": "user"}},
    {"$project": {"sentiment": "$messages.sentiment"}}
])

# 提取情感并统计频数
sentiments = [msg['sentiment'] for msg in user_messages if 'sentiment' in msg]
sentiment_counts = Counter(sentiments)

# 输出结果
print(sentiment_counts)

Counter({'中立': 17716, '负向': 7580, '正向': 2575})


In [4]:
# 定义情感映射
sentiment_mapping = {
    '中立': '中立', 'neutral': '中立', 'Neutral': '中立', 'N/A': '中立', '': '中立',
    '负向': '负向', 'negative': '负向', 'Negative': '负向', 'negaitve': '负向', '負向': '负向',
    '正向': '正向', 'positive': '正向', 'Positive': '正向', '积极': '正向',
    # 其他情感类型可以根据需要进行归类
    'frustrated': '负向', 'concerned': '负向', 'disappointed': '负向',
    'apologetic': '负向', 'confused': '中立', '请求': '中立', '要求': '中立',
    'hopeful': '正向', '幽默': '中立', '疑惑': '中立', '疑问': '中立', 'affirmative': '正向'
}

# 更新数据库中的情感数据
for message in conversations_collection.find({"messages.type": "user"}):
    updated_messages = []
    for msg in message['messages']:
        if msg['type'] == 'user' and 'sentiment' in msg:
            original_sentiment = msg['sentiment']
            # 映射情感
            msg['sentiment'] = sentiment_mapping.get(original_sentiment, '中立')
        updated_messages.append(msg)
    
    # 更新数据库
    conversations_collection.update_one(
        {"_id": message["_id"]},
        {"$set": {"messages": updated_messages}}
    )
